In [1]:
!git clone https://github.com/LokeshJatangi/Diverse_specific_clarification_questions

Cloning into 'Diverse_specific_clarification_questions'...
remote: Enumerating objects: 306, done.
remote: Counting objects: 100% (306/306), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 306 (delta 73), reused 298 (delta 71), pack-reused 0
Receiving objects: 100% (306/306), 16.59 MiB | 19.33 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [2]:
cd /content/Diverse_specific_clarification_questions


/content/Diverse_specific_clarification_questions


In [3]:
pip install -r requirements.txt  

     |████████████████████████████████| 90 kB 3.2 MB/s 


In [4]:
!wget https://zenodo.org/record/4587504/files/KPCNet_resources.zip.001
!wget https://zenodo.org/record/4587504/files/KPCNet_resources.zip.002
!wget https://zenodo.org/record/4587504/files/KPCNet_resources.zip.003
!wget https://zenodo.org/record/4587504/files/KPCNet_resources.zip.004

!cat KPCNet_resources.zip.00* > KPCNet_resources.zip
!unzip KPCNet_resources.zip

--2022-04-01 18:49:51--  https://zenodo.org/record/4587504/files/KPCNet_resources.zip.001
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67108864 (64M) [application/octet-stream]
Saving to: ‘KPCNet_resources.zip.001’

KPCNet_resources.zi 100%[===================>]  64.00M  19.2MB/s    in 3.3s    

2022-04-01 18:49:56 (19.2 MB/s) - ‘KPCNet_resources.zip.001’ saved [67108864/67108864]

--2022-04-01 18:49:56--  https://zenodo.org/record/4587504/files/KPCNet_resources.zip.002
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67108864 (64M) [application/octet-stream]
Saving to: ‘KPCNet_resources.zip.002’

KPCNet_resources.zi 100%[===================>]  64.00M  16.0MB/s    in 4.0s    

2022-04-01 18:50:02 (16.0 MB/s) - ‘KPCNet_resources.zi

In [5]:
import argparse
import os
import pickle as p
import json

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from read_data import *
from process_data import *
from run import *
from constants import *
from hparams import hparams
from utils import *
import _locale
_locale._getdefaultlocale = (lambda *args: ['en_US', 'utf8'])

In [6]:
word_embeddings = p.load(open("./data/word_embeddings.p", 'rb'))
word_embeddings = np.array(word_embeddings)
word2index = p.load(open("./data/vocab.p", 'rb'))

index2kwd, kwd2index, index2cnt = read_kwd_vocab("./data/train_kwd_vocab.txt")

index2word = reverse_dict(word2index)

In [7]:
encoder = EncoderRNN(hparams.HIDDEN_SIZE, word_embeddings, hparams.RNN_LAYERS,
                         dropout=hparams.DROPOUT, update_wd_emb=hparams.UPDATE_WD_EMB)
decoder = AttnDecoderRNN(hparams.HIDDEN_SIZE, len(word2index), word_embeddings, hparams.ATTN_TYPE,hparams.RNN_LAYERS, dropout=hparams.DROPOUT, update_wd_emb=hparams.UPDATE_WD_EMB,
                             condition=hparams.DECODER_CONDITION_TYPE)
kwd_predictor = get_predictor(word_embeddings, hparams)
kwd_bridge = MLPBridge(hparams.HIDDEN_SIZE, hparams.MAX_KWD, hparams.HIDDEN_SIZE, len(word_embeddings[0]),
                               norm_type=hparams.BRIDGE_NORM_TYPE, dropout=hparams.DROPOUT)
if hparams.USE_CUDA:
    encoder.cuda()
    decoder.cuda()
    kwd_predictor.cuda()
    kwd_bridge.cuda()

In [8]:
models = torch.load("./ckpt/s2s_D0.3_cnn_noneg_dropout_replace_fr.epoch59.models")
hparams.load("./hparams/s2s_D0.3_cnn_noneg_dropout_replace_fr.json")

In [9]:
with open("./data/kwd_filter_dict.json", encoding="utf-8") as f:
    filter_dict = json.load(f)

def make_filter_mask(post, filter_dict, kwd2index):
    curr_kwd_filter_mask = [0 for i in range(len(kwd2index))]
    for keys, to_filters in filter_dict.items():
        if keys.startswith("@") and keys.endswith("@"):  # regex
            if bool(re.search(keys[1:-1], post)):
                for kwd0 in to_filters:
                    curr_kwd_filter_mask[kwd2index[kwd0]] = -1e20
        else:
            for k in keys.split(","):
                if k in post:
                    for kwd0 in to_filters:
                        curr_kwd_filter_mask[kwd2index[kwd0]] = -1e20
    return curr_kwd_filter_mask

In [10]:
encoder.load_state_dict(models["encoder"])
decoder.load_state_dict(models["decoder"])
kwd_predictor.load_state_dict(models["kwd_predictor"])
kwd_bridge.load_state_dict(models["kwd_bridge"])

<All keys matched successfully>

In [11]:
from beam import evaluate_beam

In [12]:
def clean_html(text):
    text = re.sub(r"& (\S+) ;", r"&\1;", text)
    text = re.sub(r"& # (\S+) ;", r"&#\1;", text)
    text = html.unescape(text)
    return text

def clean_text(text):
    return re.sub(r"( <EOS>)", "", clean_html(text.strip()))

# Jaccard
def sent_sim(text1, text2):
    words1, words2 = set(text1.lower().strip().split()), set(text2.lower().strip().split())
    return len(words1 & words2) / len(words1 | words2)

def deduplicate(texts0, preserve=3, threshold=0.5):
    texts = texts0[:]
    assert len(texts) >= preserve and preserve > 0
    if len(texts) == preserve:
        return list(range(len(texts))), texts
    sel_ids, remain_ids = [0], list(range(1, len(texts)))
    sel_texts, remain_texts = texts[:1], texts[1:]
    for i in range(1, preserve):
        overlaps = []
        sel_cand = None
        for cand_id, cand in enumerate(remain_texts):
            overlap = max(sent_sim(cand, sel) for sel in sel_texts)
            if overlap < threshold:
                sel_cand = cand_id
                break
            overlaps.append(overlap)
        if sel_cand is None:
            sel_cand = np.argmin(overlaps)
        sel_texts.append(remain_texts[sel_cand])
        sel_ids.append(remain_ids[sel_cand])
        del remain_texts[sel_cand]
        del remain_ids[sel_cand]
    return sel_ids, sel_texts


In [13]:
sent0 = "sleep innovations 4-inch dual layer mattress topper . 10-year limited warranty . made in the usa . queen size  ."
words0 = sent0.strip().lower().split()[:hparams.MAX_POST_LEN]
# batch of size 1
input_seqs = [[word2index[x] if x in word2index else word2index[UNK_token] for x in words0 ]]
input_lens = [len(words0)]
test_data = [["id0"],input_seqs,input_lens,[None],[None],[0],[0]]

kwd_filter_mask0 = make_filter_mask(sent0, filter_dict, kwd2index)
kwd_filter_masks = [kwd_filter_mask0]  # the mask here is for filter out kwds
test_data[-1] = kwd_filter_masks

hparams.BATCH_SIZE = 1
out_seqs = evaluate_beam(word2index, index2word, encoder, decoder, kwd_predictor, kwd_bridge, test_data, hparams.MAX_QUES_LEN, "./infer_out", "infer", index2kwd, save_all_beam=True, infer=True)
print(out_seqs)

hparams.KWD_CLUSTERS = 2
kwd_edge_cnt = scipy.sparse.load_npz("./data/kwd_edges.npz")
kwd_clusters = get_cluster_kwds(kwd_predictor, test_data, kwd_edge_cnt, index2kwd, kwd2index)

hparams.DECODE_USE_KWD_LABEL = True
out_seqs = []
for i in range(hparams.KWD_CLUSTERS):
    test_data[5] = kwd_clusters[i]
    tmp_seqs = evaluate_beam(word2index, index2word, encoder, decoder, kwd_predictor, kwd_bridge, test_data, hparams.MAX_QUES_LEN, "./infer_out", "infer", index2kwd, save_all_beam=True, infer=True)
    print(tmp_seqs)
    out_seqs.extend(tmp_seqs)

cleaned_out_seqs = [clean_text(x) for x in out_seqs]
filtered_ids, filtered_texts = deduplicate(cleaned_out_seqs)
print(filtered_texts)


BATCH:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/content/Diverse_specific_clarification_questions/beam.py:223: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  if word2index[EOS_token] in prev_backtrack_seqs[b, topi[b][k]//decoder.output_size, :t]:
/content/Diverse_specific_clarification_questions/beam.py:228: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc

['what are the dimensions of this topper ? <EOS>\n', 'what is the warranty on this topper ? <EOS>\n', 'what are the dimensions of the bed ? <EOS>\n', 'what are the dimensions of the mattress ? <EOS>\n', 'what is the length of the bed ? <EOS>\n', 'what is the height of the bed ? <EOS>\n']


BATCH:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/content/Diverse_specific_clarification_questions/beam.py:223: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  if word2index[EOS_token] in prev_backtrack_seqs[b, topi[b][k]//decoder.output_size, :t]:
/content/Diverse_specific_clarification_questions/beam.py:228: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc

['what are the dimensions of this topper ? <EOS>\n', 'what is the warranty on this topper ? <EOS>\n', 'what are the dimensions of the bed ? <EOS>\n', 'what are the dimensions of the mattress ? <EOS>\n', 'what is the height of this topper ? <EOS>\n', 'how long is the mattress ? <EOS>\n']


BATCH: 100%|██████████| 1/1 [00:00<00:00,  3.33it/s]

['what is the thickness of the memory foam ? <EOS>\n', 'is it made in the usa ? <EOS>\n', 'what is the density of the foam ? <EOS>\n', 'what is the thickness of the foam ? <EOS>\n', 'is it made in china ? <EOS>\n', 'what is the thickness of the memory foam topper ? <EOS>\n']
['what are the dimensions of this topper ?', 'what is the warranty on this topper ?', 'how long is the mattress ?']


In [ ]:
N